# Import

In [1]:
import pandas as pd
from matplotlib import pyplot as plt

# Explore and Cleaning data

**Examine the data**

In [2]:
df = pd.read_csv("data/Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


**Number of rows and columns**

In [3]:
df.shape

(13320, 9)

**Number of each area_type**

In [4]:
df.groupby("area_type")["area_type"].count()

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8790
Name: area_type, dtype: int64

**Drop unnecessary column**
- We just need `location`, `size`, `total_sqft`, `balcony` and `price`.\
$\to$ drop the others.

In [5]:
df.drop(['area_type', 'society', 'balcony', "availability"], axis=1, inplace=True)
df.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


- Number of missing values.

In [6]:
(df.isnull().sum() / df.shape[0]) * 100

location      0.007508
size          0.120120
total_sqft    0.000000
bath          0.548048
price         0.000000
dtype: float64

- We can drop all rows that have missing value since it was very **small**.

In [7]:
df.dropna(axis=0, inplace=True)
df.shape

(13246, 5)

- Examine `size` column.

In [8]:
df["size"].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

- we just to `integer part`\
$\to$ create a column that contain `integer part` of `size` column.

In [9]:
df["BHK"] = df["size"].apply(lambda x: int(x.split(" ")[0]))
df.BHK

0        2
1        4
2        3
3        3
4        2
        ..
13315    5
13316    4
13317    2
13318    4
13319    1
Name: BHK, Length: 13246, dtype: int64

In [10]:
df.head()

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


**BHK is number of bedrooms**

In [11]:
df.BHK.unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

***There are some house that have 27 or 43 bedrooms.***

In [12]:
df[df.BHK > 20]

,location,size,total_sqft,bath,price,BHK
1718,2Electronic City Phase II,27 BHK,8000,27.0,230.0,27
4684,Munnekollal,43 Bedroom,2400,40.0,660.0,43


- Now we examine `total_sqft`

In [13]:
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      dtype=object)

We can see that `dtype` is `object` and it has value like `'1133 - 1384'`.

- Take out all rows that is not `float` in `total_sqft` column.

In [14]:
def is_float(row):
    try:
        float(row)
    except:
        return False
    return True

In [15]:
df[~df['total_sqft'].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,BHK
30,Yelahanka,4 BHK,2100 - 2850,4.0,186.000,4
122,Hebbal,4 BHK,3067 - 8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042 - 1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145 - 1340,2.0,43.490,2
188,KR Puram,2 BHK,1015 - 1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195 - 1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120 - 1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090 - 5002,4.0,445.000,4


In [16]:
df[~df['total_sqft'].apply(is_float)].tail(10)

,location,size,total_sqft,bath,price,BHK
12544,Hebbal,4 BHK,4000 - 4450,6.0,359.000,4
12560,Hosa Road,3 BHK,142.84Sq. Meter,3.0,110.000,3
12652,Billamaranahalli,2 Bedroom,300Sq. Yards,2.0,150.000,2
12791,Bannerghatta Road,2 BHK,1115 - 1130,2.0,61.740,2
12955,Thanisandra,3 BHK,1437 - 1629,3.0,75.885,3
12975,Whitefield,2 BHK,850 - 1060,2.0,38.190,2
12990,Talaghattapura,3 BHK,1804 - 2273,3.0,122.000,3
13059,Harlur,2 BHK,1200 - 1470,2.0,72.760,2
13265,Hoodi,2 BHK,1133 - 1384,2.0,59.135,2
13299,Whitefield,4 BHK,2830 - 2882,5.0,154.500,4


This column have multiple pattern of values.

We must to clean this column:
- if it have format like `123-232` $\to$ take average.
- otherwise fill it by `None`.

In [17]:
def conv2Float(row):
    tokens = row.split("-")
    
    if len(tokens) == 2:
        return (float(tokens[0]) + float(tokens[1])) / 2
    try:
        return float(row)
    except:
        return None

In [18]:
df["total_sqft"] = df["total_sqft"].apply(conv2Float)

In [19]:
df.head(10)

,location,size,total_sqft,bath,price,BHK
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2
5,Whitefield,2 BHK,1170.0,2.0,38.00,2
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6


---

# Feature Engineering

Create a column `price_per_sqft` is division `price` and `total_sqft`, but `price` in Indian is calculated in **rupees** and in our situation is calculated in **lakh rupees** which mean we should multiply is by $1,00,000$

<font color='green'>$\to$**The ratio always better than original value. </font><br>**
    *Example: when you make a cake, a ratio of sugar will give you a better insight than original value.*

In [20]:
df['price_per_sqft'] = df['price']*100000 / df['total_sqft']
df.head()

,location,size,total_sqft,bath,price,BHK,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


Examine Categorical column (i.e `location` column)

In [21]:
# get number of unique value in location column
df.location.nunique()

1304

$\to$ it quite big so we need to find the way to **reduce dimensions to avoid overfitting**. And the way to do it is remove some of sample.

While number of unique value in `location` column is **1304**, but there are some of value that only have a small number of sample $\to$ so we can all of that sample to a **whole new category**.

In [22]:
df.location = df.location.apply(lambda x: x.strip())

In [23]:
location_stats = df.groupby('location')['location'].agg('count').sort_values(ascending=False).to_frame()
location_stats.head(10)

,location
location,
Whitefield,535
Sarjapur Road,392
Electronic City,304
Kanakpura Road,266
Thanisandra,236
Yelahanka,210
Uttarahalli,186
Hebbal,176
Marathahalli,175


In [24]:
location_stats.tail(10)

,location
location,
Kamdhenu Nagar,1
Kanakadasa Layout,1
Kanakapur main road,1
Kanakapura Rod,1
Kanakapura Main Road,1
1 Giri Nagar,1
"Kanakapura Road,",1
Kanakapura main Road,1
Karnataka Shabarimala,1


We need to find which location have less or equal to 10 data points.

In [25]:
location_stats_less_than_10 = location_stats[location_stats.location <= 10]
location_stats_less_than_10

,location
location,
Basapura,10
1st Block Koramangala,10
Gunjur Palya,10
Kalkere,10
Sector 1 HSR Layout,10
...,...
1 Giri Nagar,1
"Kanakapura Road,",1
Kanakapura main Road,1


Number of location that have less or equal to 10 data points.

In [26]:
len(location_stats_less_than_10)

1052

Now we need to change name of all location that have less or equal to 10 data points to a new name **other**.

In [27]:
df.location = df.location.apply(lambda x: 'other' if x in location_stats_less_than_10.index else x)

In [28]:
df.head(10)

,location,size,total_sqft,bath,price,BHK,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3247.863248
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7467.057101
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18181.818182
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,36274.509804


In [29]:
df.location.nunique()

242

---

# Outliers Removal
We have many ways to remove Outliers from your data, it can be done by using **Knowledge Domain** (i.e house with 2 bedrooms will have total square > 600) and using **Statistic** (i.e use standard deviation).

**Using Knowledge Domain**
> Typically, 1 bedroom will place 300 square feet<br>
> So we will remove all data point that not satisfy this constraint.

In [30]:
df = df[df.total_sqft/df.BHK >= 300]
df.head()

,location,size,total_sqft,bath,price,BHK,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [31]:
# Size of dataframe after removed some inconsistent data points
len(df)

12456

**Using Statistic**
> It's about $68$% of your dataframe lie in range <font color='red'>**muy - std $\to$ muy + std**</font>.<br>
> So all data points lie outside this range will be outliers.

In [33]:
df.price_per_sqft.describe()

count     12456.000000
mean       6308.502826
std        4168.127339
min         267.829813
25%        4210.526316
50%        5294.117647
75%        6916.666667
max      176470.588235
Name: price_per_sqft, dtype: float64